In [129]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("solutions").getOrCreate()

In [130]:
from pyspark.sql.functions import sum , col ,length

In [131]:
#load datasets 

data_path = "./data/revised_final_data1.csv"  
df = spark.read.csv(data_path, header=True, inferSchema=True)

data_path = "./data/goods_classification.csv"  
df_goods = spark.read.csv(data_path, header=True, inferSchema=True)

data_path = "./data/country_classification.csv"  
df_country = spark.read.csv(data_path, header=True, inferSchema=True)
data_path = "./data/services_classification.csv"  
df_services = spark.read.csv(data_path, header=True, inferSchema=True)



In [139]:
df.select('code').where(length(col('code'))>2).show()

+----+
|code|
+----+
|0101|
|0101|
|0101|
|0101|
|0101|
|0102|
|0104|
|0104|
|0104|
|0104|
|0104|
|0105|
|0105|
|0105|
|0105|
|0105|
|0105|
|0105|
|0105|
|0105|
+----+
only showing top 20 rows



In [115]:
df.printSchema()

root
 |-- time_ref: date (nullable = true)
 |-- account: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- value: double (nullable = true)
 |-- status: string (nullable = true)



In [116]:
df = df.withColumn('value', col('value').cast('int'))

In [117]:
df.printSchema()

root
 |-- time_ref: date (nullable = true)
 |-- account: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- status: string (nullable = true)



<h2>Question1 : Calculate the total export value for each country in the 'revised_final' table. Show the top 10 countries with the highest total export values.
 </h2>

In [118]:

result = df.where(col('account')=='Exports').groupBy('country_code').agg(sum('value').alias('total_export_value'))
result = result.join(df_country,result['country_code']==df_country['country_code'],'inner')
result = result.select('country_label',df['country_code'],'total_export_value')
result = result.orderBy(result.total_export_value.desc()).limit(10)
result.show()

+--------------------+------------+------------------+
|       country_label|country_code|total_export_value|
+--------------------+------------+------------------+
|China, People's R...|          CN|      168541418942|
|           Australia|          AU|      124044791511|
|United States of ...|          US|      106261562532|
|               Japan|          JP|       40647876106|
|  Korea, Republic of|          KR|       24767856475|
|      United Kingdom|          GB|       22290125277|
|           Singapore|          SG|       18358723868|
|           Indonesia|          ID|       15861041776|
|              Taiwan|          TW|       15548494077|
|Hong Kong (Specia...|          HK|       12708136935|
+--------------------+------------+------------------+

